### Tracking objects using color spaces

In [1]:
import cv2
import numpy as np

In [2]:
#Define a function to get the current frame from the video capture object
def get_frame(cap, scaling_factor):
    #Read the current frame from the video capture object
    _, frame = cap.read()
    
    #Resize the image
    frame = cv2.resize(frame, None, fx = scaling_factor, fy = scaling_factor, interpolation = cv2.INTER_AREA)
    
    return frame

In [3]:
def start_video_capture():
    cap = cv2.VideoCapture('data/video_sugar_the_movie.mp4')
    
    #Define the scaling factor for the images
    scaling_factor = 0.5
    
    #Keep reading the frames from the video capture object until the user hits the 'Esc' key
    while True:
        #Grab the current frame
        frame = get_frame(cap, scaling_factor)
        
        #Convert the image to HSV colorspace
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # Define range of skin color in HSV
        lower = np.array([0, 70, 60])
        upper = np.array([50, 150, 255])
        
        #Threshold the HSV image to only get cyan blue color
        mask = cv2.inRange(hsv, lower, upper)
        
        #Compute the Bitwise-AND between the mask and the original image
        img_bitwise_and = cv2.bitwise_and(frame, frame, mask = mask)
        
        #Run median blurring to smoothen the image
        img_median_blurred = cv2.medianBlur(img_bitwise_and, 5)
        
        #Display the input and output
        cv2.imshow('Input', frame)
        cv2.imshow('Output', img_median_blurred)
        
        #Check if the user hits the 'Esc' key
        stop_key = cv2.waitKey(5)
        if stop_key == 27:
            break
            
    #Closing all windows properly
    cv2.destroyAllWindows()

In [22]:
start_video_capture()

### Object tracking using background subtraction

In [29]:
x = cv2.createBackgroundSubtractorMOG2()
x.apply?

In [30]:
def start_video_capture_bg_subtract():
    # Define the video capture object
    cap = cv2.VideoCapture('data/video_sugar_the_movie.mp4')
    
    #Define the background subtractor object
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    
    # Define the number of previous frames to use to learn. This factor controls the learning rate of the algorithm.
    # The learning rate refers to the rate at which your model will learn about the background. 
    # Higher value for 'history' indicates a slower learning rate.
    history = 100
    learning_rate = 1/history #Define the learning rate
    
    scaling_factor = 0.5
    
    #Keep reading the frames from the video capture object until the user hits the 'Esc' key
    while True:
        #Grab the current frame
        frame = get_frame(cap, scaling_factor)
        
        #Compute the mask
        mask = bg_subtractor.apply(frame, learningRate = learning_rate)
        
        #Convert the mask from grayscale to rgb
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        
        #Display the images
        cv2.imshow('Input', frame)
        cv2.imshow('Output', frame & mask)
        
        #Check if the user hits the 'Esc' key
        stop_key = cv2.waitKey(10)
        if stop_key == 27:
            break
            
    #Release the video capture object
    cap.release()
    #Close all windows
    cv2.destroyAllWindows()

In [31]:
start_video_capture_bg_subtract()

### Building an interactive object tracker using the CAMShift algorithm

In [19]:
#Define a class to handle object tracking-related functionality
class ObjectTracker(object):
    def __init__(self, scaling_factor = 0.5):
        #Initialize the video capture object
        self.cap = cv2.VideoCapture('data/video_messi.mp4')
        
        #Capture the frame from the video capture object
        _, self.frame = self.cap.read()
        
        self.scaling_factor = scaling_factor #Scaling factor for the captured frame
        
        #Resize the frame
        self.frame = cv2.resize(self.frame, None, fx = self.scaling_factor, fy = self.scaling_factor, 
                                interpolation = cv2.INTER_AREA)
        
        #Create a window to display the frame
        cv2.namedWindow('Object Tracker')
        
        #Set the mouse callback function to track the mouse
        cv2.setMouseCallback('Object Tracker', self.mouse_event)
        
        #Initialize variables to track the rectangular section for the object to be tracked
        self.selection = None   #initialize variable related to rectangular region selection
        self.drag_start = None   #Initialize variable related to starting position
        self.tracking_state = 0   #Initialize variable related to the state of tracking
       
    
    def mouse_event(self, event, x, y, flags, param):
        #Define a method to track mouse events
        x, y = np.int16([x, y])   #Convert x and y coordinates into 16-bit numpy integers
        
        #Check if a mouse button down event has occured since that should indicate the user has started drawing a rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.tracking_state = 0
            
        #Check if the user has started dragging the mouse, to select the region
        if self.drag_start:
            if flags & cv2.EVENT_FLAG_LBUTTON:
                #Extract the dimensions of the frame
                h, w = self.frame.shape[:2]
                
                #Get the initial position
                xi, yi = self.drag_start
                
                #Get the max and min values to make it agnostic to the direction that the mouse is dragged to
                x0, y0 = np.maximum(0, np.minimum([xi, yi], [x, y]))
                x1, y1 = np.minimum([w, h], np.maximum([xi, yi], [x, y]))
                
                #Reset the selection variable
                self.selection = None
                
                #Finalize the rectangular selection
                if (x1 - x0 > 0) and (y1 - y0 > 0):
                    self.selection = (x0, y0, x1, y1)
            
            else:
                #If the selection is done, start tracking
                self.drag_start = None
                if self.selection is not None:
                    self.tracking_state = 1
        
    
    def start_tracking(self):
        #Iterate until the user hits the 'Esc' key
        while True:
            #Capture the frame from the video capture object
            _, self.frame = self.cap.read()
            
            #Resize the input frame
            self.frame = cv2.resize(self.frame, None, fx = self.scaling_factor, fy = self.scaling_factor,
                                   interpolation = cv2.INTER_AREA)
            
            #Create a copy of the frame, which will be used later in the code
            vis = self.frame.copy()
            
            #Convert the frame to HSV Colorspace
            hsv = cv2.cvtColor(self.frame, cv2.COLOR_BGR2HSV)
            
            #Create the mask based on predefined thresholds
            mask = cv2.inRange(hsv, np.array([0., 60., 32.]), np.array([180., 255., 255.]))
            
            #Check if the user has selected the region
            if self.selection:
                #Extract the coordinates of the selected rectangle
                x0, y0, x1, y1 = self.selection
                
                #Extract the tracking window
                self.track_window = (x0, y0, x1-x0, y1-y0)
                
                #Extract the regions of interest
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                
                #Compute the histogram of the region of interest in the HSV image using the mask
                hist = cv2.calcHist([hsv_roi], [0], mask_roi, [16], [0, 180])
                
                #Normalize and reshape the histogram
                cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                
                #Extract the region of interest from the frame
                vis_roi = vis[y0:y1, x0:x1]
                
                #Compute the image negative (for display only)
                cv2.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0
                
            #Check if the system is in the 'tracking' mode
            if self.tracking_state == 1:
                self.selection = None   #Reset the selection variable
                
                #Compute the histogram back projection
                hsv_backproj = cv2.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                
                #Compute bitwise AND between histogram
                hsv_backproj &= mask
                
                #Define termination criteria for the tracker
                term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
                
                #Apply the CAMShift algorithm on the back project histogram
                track_box, self.track_window = cv2.CamShift(hsv_backproj, self.track_window, term_crit)
                
                #Draw an ellipse around the object
                cv2.ellipse(vis, track_box, (0, 255, 0), 2)
            
            #Show the output live video
            cv2.imshow('Object Tracker', vis)
        
            #Stop if the user hits the 'Esc' key
            stop_key = cv2.waitKey(5)
            if stop_key == 27:
                break
        
        #Close all windows
        cv2.destroyAllWindows()

In [28]:
ObjectTracker().start_tracking()